In [1]:
import pandas as pd
import numpy as np
import os
# import pickle
from tqdm import tqdm
import re
from collections import Counter
import operator


In [2]:
def get_columns():
    columns = pd.read_csv("column_names.csv" ,header=None).values[0]
    columns = list(columns)
    print(columns)

    return columns

In [3]:
get_columns()

['tweet_id', 'username', 'timestamp', '#followers', '#friends', '#retweets', '#favourites', 'entities', 'sentiment', 'mentions', 'hashtags', 'urls']


['tweet_id',
 'username',
 'timestamp',
 '#followers',
 '#friends',
 '#retweets',
 '#favourites',
 'entities',
 'sentiment',
 'mentions',
 'hashtags',
 'urls']

In [4]:
df = pd.read_csv('TweetsCOV19.tsv', sep='\t', error_bad_lines=False,  names = get_columns())

['tweet_id', 'username', 'timestamp', '#followers', '#friends', '#retweets', '#favourites', 'entities', 'sentiment', 'mentions', 'hashtags', 'urls']


In [5]:
df.shape[0]

8077794

# Hashtag popularity
How many times a hashtag in a tweet appeared in all tweets

In [6]:
def get_top_hashtags(df):
    colnames = ['tweet_id','hashtags']
    df2 = df[colnames]

    df3 = df2
    df3['hashtags'] = df2['hashtags'].str.lower()
    df3['hashtags'] = df3['hashtags'].astype(str)

    hashtag_list = df3['hashtags'].tolist()
    hashtag = []
    for i in tqdm(range(len(hashtag_list))):
        hashtag_all = ''.join(hashtag_list[i])
        hashtag_all_clean = re.sub(r'[^\w]', ' ', hashtag_all)
        hashtag_null_removed = hashtag_all_clean.replace("null", "")
        hashtag_under_removed = hashtag_null_removed.replace("_", "")
        hashtag_dash_removed = hashtag_under_removed.replace("-", "")
        hashtag_longdash_removed = hashtag_dash_removed.replace("ー", "")
        hashtag.append(hashtag_longdash_removed.split(' '))
        
    hashtag_all = ' '.join(hashtag_list)
    hashtag_all_clean = re.sub(r'[^\w]', ' ', hashtag_all)
    hashtag_null_removed = hashtag_all_clean.replace("null", "")
    hashtag_under_removed = hashtag_null_removed.replace("_", "")
    hashtag_dash_removed = hashtag_under_removed.replace("-", "")
    hashtag_longdash_removed = hashtag_dash_removed.replace("ー", "")
    hashtag_list = hashtag_longdash_removed.split(' ')
    hashtag_counter = Counter(hashtag_list)

    dd = dict(sorted(hashtag_counter.items(), key=operator.itemgetter(1),reverse=True))
    #manual cleaning
    dd.pop('')
    dd.pop('1')
    dd.pop('2')
    dd.pop('0')
    dd.pop('2020')
    dd.pop('0000')
    dd.pop('s')
    dd.pop('19')
    dd.pop('3')
    dd.pop('29')
    dd.pop('21')
    
    
    k = Counter(dd)
    high = k.most_common(20) 
    top20hashtag_list = []
    for i in range(len(high)):
        top20hashtag_list.append(high[i][0])
    

    top20hashtagcol = []
    for i in tqdm(range(len(hashtag))):
        counter = 0
        for j in range(len(hashtag[i])):
            if hashtag[i][j] in top20hashtag_list:
                counter += 1
        top20hashtagcol.append(counter)
        
    df['hashtag_popularity'] = top20hashtagcol
    
    return df

In [7]:
df_new = get_top_hashtags(df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|████████████████████████████████████████████████████████████████████| 8077794/8077794 [00:09<00:00, 837314.19it/s]


In [81]:
df_new

,tweet_id,username,timestamp,#followers,#friends,#retweets,#favourites,entities,sentiment,mentions,hashtags,urls,hashtag_popularity
0,1178791787386814465,35234fe4a19cc1a3336095fb3780bcc1,Mon Sep 30 22:00:37 +0000 2019,619,770,0,0,null;,2 -1,null;,null;,null;,0
1,1178791985106153472,ea4592f39636d87af8fb4b17b7e2e4c0,Mon Sep 30 22:01:24 +0000 2019,36365,19344,13,17,nazi:Nazism:-2.742538749414189;blood money:Blo...,1 -4,null;,null;,https://twitter.com/himalayahawk/status/117766...,0
2,1178793230223183872,bf05d1888dd974fa4a8679c25e2ead03,Mon Sep 30 22:06:21 +0000 2019,5018,1933,0,0,vaccine:Vaccine:-2.6651530673745762;anti vaxxe...,2 -1,null;,null;,https://goo.gl/fb/uoeiPk:-:,0
3,1178795172206919680,eb8a99bca8945eab1d006750e9b75518,Mon Sep 30 22:14:04 +0000 2019,2219,971,0,0,muse:Muse_%28band%29:-2.1677823918620867;talki...,2 -1,null;,null;,null;,0
4,1178798309491822592,00695cf79d60e86a5d0f872f4358337a,Mon Sep 30 22:26:32 +0000 2019,9009,4943,26,89,people of northern ireland:People_of_Northern_...,2 -1,null;,VFS,null;,0
5,1178801459439296514,6ff6c6532fe0c399dcedd1dad6249c28,Mon Sep 30 22:39:03 +0000 2019,284,935,0,0,cnn:CNN:-0.9113024985533761;,3 -2,null;,null;,null;,0
6,1178802464914657286,dbd298dfc487b7e6a7e220439dc91983,Mon Sep 30 22:43:03 +0000 2019,3605,3526,5,13,null;,3 -1,null;,null;,null;,0
7,1178803230962327555,c3df67279d13e650c44d529c92476d30,Mon Sep 30 22:46:06 +0000 2019,5945,47,1,6,youtube:YouTube:-1.0295623705363122;super bowl...,3 -1,YouTube,null;,https://www.youtube.com/watch?v=jWdhOfNmLsE&fe...,0
8,1178803487037108225,fc48c9cc30420a1af599e735a14b0b8a,Mon Sep 30 22:47:07 +0000 2019,156538,3858,56,131,vontaze burfict:Vontaze_Burfict:-0.77486020920...,1 -1,null;,Raiders,null;,0
9,1178803977376415749,f734f2f2c94d7de900d18c59cc858d48,Mon Sep 30 22:49:03 +0000 2019,27014,24488,39,37,winter storm:Winter_storm:-1.9824755199970392;...,2 -1,AOC,null;,https://mtstandard.com/news/local/winter-storm...,0


# Entitity popularity
How many times an entity in a tweet appeared in all tweets|

In [22]:
df["entities"] = df["entities"].astype(str)
df["entities"] = df["entities"].apply(lambda x: x.lower())

def entities_splitter(x):
    entity_list = []
    that_entity_list = x.split(";")
    for counter in range(0, len(that_entity_list)-1):
        each_entity_list = that_entity_list[counter].split(":")
        that_entity = each_entity_list[0]
        if that_entity == "null":
            pass
        else:
            entity_list.append(that_entity)
    return(entity_list)

df["entities_list"] = df["entities"].apply(entities_splitter)

In [31]:
from itertools import chain

def get_top_entities(df):
    # required to get entities_list column from entities_splitter function 
    entities_list = df['entities_list'].tolist()
    entities_all = list(chain(*entities_list))
    entities_counter = Counter(entities_all)
    dd = dict(sorted(entities_counter.items(), key=operator.itemgetter(1),reverse=True))
    k = Counter(dd)
    high = k.most_common(20) 

    top20entities_list = []
    for i in range(len(high)):
        top20entities_list.append(high[i][0])


    top20entitiescol = []
    for i in tqdm(range(len(entities_list))):
        counter = 0
        for j in range(len(entities_list[i])):
            if entities_list[i][j] in top20entities_list:
                counter += 1
        top20entitiescol.append(counter)

    df['entities_popularity'] = top20entitiescol
    return df


100%|████████████████████████████████████████████████████████████████████| 8077794/8077794 [00:09<00:00, 884943.91it/s]


In [32]:
df[df['entities_popularity']==4]

,tweet_id,username,timestamp,#followers,#friends,#retweets,#favourites,entities,sentiment,mentions,hashtags,urls,hashtag_popularity,entities_list,No. of top 20 Entities,entities_popularity
2622,1181236818597027844,e5899ff8c224af8c33556af7d779d2db,Mon Oct 07 15:56:18 +0000 2019,494578,126860,155,407,china:china:-2.113921624336916;china:china:-2....,1 -2,null;,null;,null;,0,"[china, china, china, china, gop, wise]",0,4
6020,1184180489822920709,f2e084f4ace15ffdc946b04927cd0b04,Tue Oct 15 18:53:24 +0000 2019,2960,5001,1,0,twitter:twitter:-1.1708686827892931;twitter:tw...,2 -4,SteveKerr 1776Stonewall,null;,http://Morey.You:-:,0,"[twitter, twitter, potus, tweet]",0,4
9185,1187081200055279616,c718d31045e30851a503e5f8f7b568a7,Wed Oct 23 18:59:47 +0000 2019,56493,57112,77,56,global warming:global_warming:-1.4269720016107...,2 -1,null;,Swamp_Busters,null;,0,"[global warming, russia, china, swamp, retweet...",0,4
11218,1189027992636481537,f8e20266297a32999381766c672971a5,Tue Oct 29 03:55:39 +0000 2019,35345,0,92,213,india:india:-1.6783115108896653;pakistan:pakis...,2 -1,null;,null;,https://www.brecorder.com/2019/10/28/538817/ch...,0,"[india, pakistan, china, china, china, fatf]",0,4
11331,1189173134374723584,9cef782e5b1c95e76cbd1810879b47f0,Tue Oct 29 13:32:23 +0000 2019,732,820,0,0,russia:russia:-1.4800050090279415;russia:russi...,2 -1,ErnestLamonica,null;,null;,0,"[russia, russia, russia, russia, wendi deng, c...",0,4
16321,1182305318476419074,658d901ea181a919aa71e1069ee63d41,Thu Oct 10 14:42:08 +0000 2019,95265,35903,133,332,iran:iran:-1.1743154944386012;russia:russia:-1...,1 -1,realDonaldTrump,null;,null;,0,"[iran, russia, russia, russia, russia, dems, d...",0,4
16645,1182561485823848448,27cd9f924a52bf60464a449286d95bf4,Fri Oct 11 07:40:03 +0000 2019,1959505,917,26,49,xi jinping:xi_jinping:-0.7756868735617743;nare...,1 -1,null;,null;,https://www.nytimes.com/2019/10/11/world/asia/...,0,"[xi jinping, narendra modi, india, india, chin...",0,4
26089,1180006022766829568,0a8adb94ace5fe74f84df2129fcbf53c,Fri Oct 04 06:25:33 +0000 2019,29977,28035,12,25,saudi arabia:saudi_arabia:-1.3281860726510886;...,1 -2,realDonaldTrump GMA GStephanopoulos,null;,null;,0,"[saudi arabia, saudi arabia, russia, russia, r...",0,4
28448,1182068453173317632,f8b703757bd7df0ddd0dc704a8b9f9e2,Wed Oct 09 23:00:55 +0000 2019,637,1007,0,0,india:india:-1.6783115108896653;china:china:-2...,2 -1,jimgeraghty neontaster,null;,null;,0,"[india, china, china, china, nba]",0,4
28759,1182324121666850816,a6fe20aeed3ad09489609fe74c7bb67f,Thu Oct 10 15:56:51 +0000 2019,227801,2105,147,262,pakistan:pakistan:-1.7781103294712668;stenogra...,3 -1,null;,XiInIndia,null;,0,"[pakistan, stenographer, doklam, china, china,...",0,4


# Mentions popularity
How many times a mentioned user in a tweet appeared in all tweets

In [56]:
def get_top_entities(df):
    df['mentions'] = df['mentions'].str.lower()
    df['mentions'] = df['mentions'].astype(str)
    mentions_list = df['mentions'].tolist()
    for i in range(len(mentions_list)):
        x = mentions_list[i].replace(" ", ",")
        mentions_list[i]= x.split(",")
        if mentions_list[i] == ['null;']:
            mentions_list[i].clear()
    from itertools import chain
    mentions_all = list(chain(*mentions_list))
    mentions_counter = Counter(mentions_all)
    dd = dict(sorted(mentions_counter.items(), key=operator.itemgetter(1),reverse=True))
    dd.pop('')
    dd.pop('+0000')

    k = Counter(dd)
    high = k.most_common(20) 

    top20mentions_list = []
    for i in range(len(high)):
        top20mentions_list.append(high[i][0])


    top20mentionscol = []
    for i in tqdm(range(len(mentions_list))):
        counter = 0
        for j in range(len(mentions_list[i])):
            if mentions_list[i][j] in top20mentions_list:
                counter += 1
        top20mentionscol.append(counter)

    df['mentions_popularity'] = top20mentionscol
    return df

In [53]:
df['mentions'][df['mentions_popularity']==3]

1098       realdonaldtrump gop housedemocrats adamschiff ...
3554       cnn realdonaldtrump fdacommissioner cdcgov hhs...
7403       joebiden realdonaldtrump, rudygiuliani cnn fox...
8113           speakerpelosi realdonaldtrump realdonaldtrump
8846       cdcgov us_fda fdatobacco fdacommissioner secaz...
9071       kris13915204 whitehouse realdonaldtrump realdo...
11084      cdcgov cdcmmwr cdcgov realdonaldtrump secazar ...
11776      asg1956 incindia rssurjewala narendramodi pmoi...
13015      msmonathomas deanobeidallah realdonaldtrump po...
14198                 potus realdonaldtrump joebiden ewarren
14383       realdonaldtrump potus senatemajldr gopleader gop
14635      danilynnguess marisal001 jacquie04063120 jaket...
15273      flotus whitehouse realdonaldtrump flotus white...
15307      realdonaldtrump flotus fdacommissioner cdcgov gop
17647      1krohmer13 snowfazam12 nytimes cnn realdonaldt...
21541      atrupar gop realdonaldtrump repadamschiff spea...
26682                   

In [54]:
df['mentions'][36597]

'potus cdcgov potus'

In [57]:
get_top_entities(df)

100%|███████████████████████████████████████████████████████████████████| 8077794/8077794 [00:06<00:00, 1181855.64it/s]


,tweet_id,username,timestamp,#followers,#friends,#retweets,#favourites,entities,sentiment,mentions,hashtags,urls,mentions_popularity
0,1178791787386814465,35234fe4a19cc1a3336095fb3780bcc1,Mon Sep 30 22:00:37 +0000 2019,619,770,0,0,null;,2 -1,null;,null;,null;,0
1,1178791985106153472,ea4592f39636d87af8fb4b17b7e2e4c0,Mon Sep 30 22:01:24 +0000 2019,36365,19344,13,17,nazi:Nazism:-2.742538749414189;blood money:Blo...,1 -4,null;,null;,https://twitter.com/himalayahawk/status/117766...,0
2,1178793230223183872,bf05d1888dd974fa4a8679c25e2ead03,Mon Sep 30 22:06:21 +0000 2019,5018,1933,0,0,vaccine:Vaccine:-2.6651530673745762;anti vaxxe...,2 -1,null;,null;,https://goo.gl/fb/uoeiPk:-:,0
3,1178795172206919680,eb8a99bca8945eab1d006750e9b75518,Mon Sep 30 22:14:04 +0000 2019,2219,971,0,0,muse:Muse_%28band%29:-2.1677823918620867;talki...,2 -1,null;,null;,null;,0
4,1178798309491822592,00695cf79d60e86a5d0f872f4358337a,Mon Sep 30 22:26:32 +0000 2019,9009,4943,26,89,people of northern ireland:People_of_Northern_...,2 -1,null;,VFS,null;,0
5,1178801459439296514,6ff6c6532fe0c399dcedd1dad6249c28,Mon Sep 30 22:39:03 +0000 2019,284,935,0,0,cnn:CNN:-0.9113024985533761;,3 -2,null;,null;,null;,0
6,1178802464914657286,dbd298dfc487b7e6a7e220439dc91983,Mon Sep 30 22:43:03 +0000 2019,3605,3526,5,13,null;,3 -1,null;,null;,null;,0
7,1178803230962327555,c3df67279d13e650c44d529c92476d30,Mon Sep 30 22:46:06 +0000 2019,5945,47,1,6,youtube:YouTube:-1.0295623705363122;super bowl...,3 -1,youtube,null;,https://www.youtube.com/watch?v=jWdhOfNmLsE&fe...,1
8,1178803487037108225,fc48c9cc30420a1af599e735a14b0b8a,Mon Sep 30 22:47:07 +0000 2019,156538,3858,56,131,vontaze burfict:Vontaze_Burfict:-0.77486020920...,1 -1,null;,Raiders,null;,0
9,1178803977376415749,f734f2f2c94d7de900d18c59cc858d48,Mon Sep 30 22:49:03 +0000 2019,27014,24488,39,37,winter storm:Winter_storm:-1.9824755199970392;...,2 -1,aoc,null;,https://mtstandard.com/news/local/winter-storm...,0
